# ArcGIS Hub Site Google Analytics To ArcGIS Online Notebook
## ***Wrote by Ben Beattie in September 2024***

This python notebook is designed to grab the Google Analytics data from [LPT AT Work ArcGIS Hub Site](https://www.lowerpaxtonatwork.com/) and upload it to ArcGIS Online for use in a Site Analytics ArcGIS Dashboard.


## Install dependent libraries:

In [22]:
!pip install google-analytics-data
!pip install geopandas
!pip install python-dotenv

DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/tflite_model_maker-0.3.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


## Import required modules:

In [23]:
import os
from dotenv import load_dotenv
import pandas as pd
from arcgis.geocoding import geocode
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.features import GeoAccessor
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
)
import geopandas as gpd
from IPython.display import display, Markdown
import zipfile
from tqdm.notebook import tqdm
from time import sleep
from datetime import datetime

## Connect Google Analytics and ArcGIS Online Connections & Initialize GIS and Analytics Clients

In [24]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= r'/arcgis/home/Credentials/credentials.json'
load_dotenv(r'/arcgis/home/Credentials/credentials.env')

username = os.getenv('arcgis_username')
password = os.getenv('arcgis_password')
url = os.getenv('arcgis_url')
property_id = os.getenv('property_id')

client = BetaAnalyticsDataClient()
gis = GIS(username=username, password=password, url=url)

## Define Functions To Create/Overwrite a Pandas DataFrame/GeoDataFrame to a Hosted Table/Feature Service in AGOL

In [25]:
def dfToHostedTable(df, title):
    cleaned_title = title.replace(" ", "_")
    df_csv = df.to_csv(f'{cleaned_title}.csv')
    display(Markdown("### 🛠️ DFToHostedTable Function Running: *(Created By Ben Beattie)*"))

    with tqdm(total=100, desc="Starting DFToHostedTable", bar_format='{desc}{bar}') as pbar:
        try:
            csv_AGOL_item = gis.content.add(item_properties={'title': cleaned_title, 'type': 'CSV'}, data=f'/arcgis/{cleaned_title}.csv')
            pbar.set_description("Uploading CSV To AGOL")
            pbar.update(50) 
            sleep(1)

            table_AGOL_item = csv_AGOL_item.publish()
            pbar.set_description("Publishing CSV As Hosted Table Service")
            pbar.update(100)
            sleep(0.5)

            display(Markdown(f"✅***{cleaned_title}** has been published with item ID: {table_AGOL_item.id}*"))

        except Exception as e:
            pbar.set_description("Table Already Exists")
            sleep(0.5)
            pbar.update(33.33)
            pbar.set_description("Seaching For Table")
            search_results = gis.content.search(f'title: "{cleaned_title}" type: "Feature Service"')

            if search_results:
                sleep(0.5)
                pbar.update(66.66)
                pbar.set_description("Table Found")
                sleep(0.5)
                pbar.set_description("Overwriting Table")
                table_AGOL_item = search_results[0]
                flc = FeatureLayerCollection.fromitem(table_AGOL_item)
                flc.manager.overwrite(f'{cleaned_title}.csv')

                display(Markdown(f"✅ ***{table_AGOL_item.title}** has been updated...*"))
                pbar.update(100)
            else:
                print("Could not find or create existing table.")
            pbar.close()

In [26]:
def gdfToFeatureService(gdf, title):
    cleaned_title = title.replace(" ", "_")
    gdf.to_file(f"/arcgis/{cleaned_title}.geojson", driver='GeoJSON', engine='fiona', crs='4326')
    display(Markdown("### 🛠️ GDFToHostedFeatureService Function Running: *(Created By Ben Beattie)*"))

    with tqdm(total=100, desc="Starting GDFToHostedTable", bar_format='{desc}{bar}') as pbar:
        search_results = gis.content.search(f'title: "{cleaned_title}" type: "GeoJson"')
        if search_results == []:
            geojson_AGOL_item = gis.content.add(item_properties={'title': f'{cleaned_title}', 'type': 'GeoJson', 'spatialReference': '4326'}, data=f'/arcgis/{cleaned_title}.geojson')
            pbar.set_description("Uploading GeoJSON To AGOL")
            pbar.update(50) 
            sleep(1)

            fs_AGOL_item = geojson_AGOL_item.publish()
            pbar.set_description("Publishing GeoJSON As Hosted Feature Service")
            pbar.update(100)
            sleep(0.5)

            display(Markdown(f"✅***{cleaned_title}** has been published with item ID: {fs_AGOL_item.id}*"))

        elif len(search_results) > 0:
            pbar.set_description("Feature Service Already Exists")
            sleep(0.5)
            pbar.update(33.33)
            pbar.set_description("Seaching For Feature Service")
            search_results = gis.content.search(f'title: "{cleaned_title}" type: "Feature Service"')
            
            if search_results:
                sleep(0.5)
                pbar.update(66.66)
                pbar.set_description("Feature Service Found")
                sleep(0.5)
                pbar.set_description("Overwriting Feature Service")
                fs_AGOL_item = search_results[0]
                flc = FeatureLayerCollection.fromitem(fs_AGOL_item)
                flc.manager.overwrite(f'/arcgis/{cleaned_title}.geojson')

                display(Markdown(f"✅ ***{fs_AGOL_item.title}** has been updated...*"))
                pbar.update(100)
            else:
                print("Could not find or create existing feature service.")
            pbar.close()

## 1. Retrieve viewer's language, output to dataframe

In [27]:
request_language = RunReportRequest(
  property=f"properties/{property_id}",
  dimensions=[
      Dimension(name="language")],
      metrics=[Metric(name="totalUsers")],
  date_ranges=[DateRange(start_date="2024-09-15", end_date="today")],
  )

response_language = client.run_report(request_language)
display(Markdown("### Google Analytics response has been received:"))
display(Markdown("#### Converting To Pandas DataFrame:"))

language_df_length = range(len(response_language.rows))

language_df = pd.DataFrame(index=(language_df_length),columns=['Total Users:', 'Language:'])

for index, row in tqdm(enumerate(response_language.rows), total=len(response_language.rows), bar_format='{desc}{bar}{n_fmt}/{total_fmt} rows'):
  language_df.iloc[index] = [
      int(row.metric_values[0].value),
      row.dimension_values[0].value
  ]

### Google Analytics response has been received:

#### Converting To Pandas DataFrame:

          0/1 rows

In [28]:
dfToHostedTable(language_df, 'LPT At Work Language Analytics Table')

### 🛠️ DFToHostedTable Function Running: *(Created By Ben Beattie)*

Starting DFToHostedTable          

✅ ***LPT_At_Work_Language_Analytics_Table** has been updated...*

## 2. Retrieve each page's views, output to dataframe

In [29]:
request_pages = RunReportRequest(
  property=f"properties/{property_id}",
  dimensions=[
      Dimension(name="pagePath")],
      metrics=[Metric(name="totalUsers")],
  date_ranges=[DateRange(start_date="2024-09-15", end_date="today")],
  )

response_pages = client.run_report(request_pages)
display(Markdown("### Google Analytics response has been received:"))
display(Markdown("#### Converting To Pandas DataFrame:"))

pages_df_length = range(len(response_pages.rows))
pages_df = pd.DataFrame(index=(pages_df_length),columns=['Total Users:', 'Page Path:'])

for index, row in tqdm(enumerate(response_pages.rows), total=len(response_pages.rows), desc="Creating DataFrame", bar_format='{desc}{bar}{n_fmt}/{total_fmt} rows'):
    pages_df.iloc[index] = [
      int(row.metric_values[0].value),
      row.dimension_values[0].value]
    sleep(0.1)

for index, row in tqdm(pages_df.iterrows(), total=pages_df.shape[0], desc="Fixing URL Paths", colour="red", bar_format='{desc}{bar}{n_fmt}/{total_fmt} rows'):
    if row['Page Path:'] == '/':
        pages_df.at[index, 'Page Path:'] = '/pages/homepage'
    if row['Page Path:'] == '/edit':
        pages_df.drop(index, inplace=True)
    sleep(0.1)

### Google Analytics response has been received:

#### Converting To Pandas DataFrame:

Creating DataFrame          0/10 rows

Fixing URL Paths          0/10 rows

In [30]:
dfToHostedTable(pages_df, 'LPT At Work Page Analytics Table')

### 🛠️ DFToHostedTable Function Running: *(Created By Ben Beattie)*

Starting DFToHostedTable          

✅ ***LPT_At_Work_Page_Analytics_Table** has been updated...*

## 3. Retrieve each device/browser used to view site, output to dataframe

In [31]:
request_device = RunReportRequest(
  property=f"properties/{property_id}",
  dimensions=[
      Dimension(name="browser"),
      Dimension(name="deviceCategory"),
      #Dimension(name="screenResolution"),
      ],
  metrics=[Metric(name="totalUsers")],
  date_ranges=[DateRange(start_date="2024-09-15", end_date="today")],
  )

response_device = client.run_report(request_device)
display(Markdown("### Google Analytics response has been received:"))
display(Markdown("#### Converting To Pandas DataFrame:"))

device_df_length = range(len(response_device.rows))
device_df = pd.DataFrame(index=(device_df_length),columns=['Total Users:', 'Browser:', 'Device Type:',
                                                           #'Screen Resolution:'
                                                           ])

for index, row in tqdm(enumerate(response_device.rows), total=len(response_device.rows), desc="Creating DateFrame", bar_format='{desc}{bar}{n_fmt}/{total_fmt} rows'):
    device_df.iloc[index] = [
        row.metric_values[0].value,
        row.dimension_values[0].value,
        row.dimension_values[1].value.capitalize(),
        #row.dimension_values[2].value
    ]
    sleep(0.1)

### Google Analytics response has been received:

#### Converting To Pandas DataFrame:

Creating DateFrame          0/5 rows

In [32]:
dfToHostedTable(device_df, 'LPT At Work Device Analytics Table')

### 🛠️ DFToHostedTable Function Running: *(Created By Ben Beattie)*

Starting DFToHostedTable          

✅ ***LPT_At_Work_Device_Analytics_Table** has been updated...*

## 4. Retrieve site views each day, output to dataframe

In [33]:
request_dateviews = RunReportRequest(
    property=f"properties/{property_id}",
    dimensions=[Dimension(name="date")],
    metrics=[Metric(name="totalUsers")],
    date_ranges=[DateRange(start_date="2024-09-15", end_date="today")],
    )

response_dateviews = client.run_report(request_dateviews)
dateviews_df_length = range(len(response_dateviews.rows))
dateviews_df = pd.DataFrame(index=(dateviews_df_length),columns=['Total Users:', 'Date:'])


for index, row in tqdm(enumerate(response_dateviews.rows), total=len(response_dateviews.rows), desc="Creating DateFrame", bar_format='{desc}{bar}{n_fmt}/{total_fmt} rows'):
    dateviews_df.iloc[index] = [
        int(row.metric_values[0].value),
        datetime.strptime(row.dimension_values[0].value, "%Y%m%d")
    ]
    sleep(0.1)

Creating DateFrame          0/9 rows

In [34]:
dfToHostedTable(dateviews_df, 'LPT At Work Time Analytics Table' )

### 🛠️ DFToHostedTable Function Running: *(Created By Ben Beattie)*

Starting DFToHostedTable          

✅ ***LPT_At_Work_Time_Analytics_Table** has been updated...*

## 5. Retrieve cities, output to GeoDataFrame, output to GeoJSON, output to Feature Service

In [35]:
request_cities = RunReportRequest(
  property=f"properties/{property_id}",
  dimensions=[
      Dimension(name="city"),
      Dimension(name="region"),
      Dimension(name="country")],
  metrics=[Metric(name="totalUsers")],
  date_ranges=[DateRange(start_date="2024-09-15", end_date="today")],
  )

response_cities = client.run_report(request_cities)
filtered_response = [row for row in response_cities.rows if "(not set)" not in [row.dimension_values[i].value for i in range(3)]]

cities_df_length = range(len(filtered_response))

cities_df = pd.DataFrame(index=(cities_df_length),columns=['Total Users:', 'City:', 'Region:', 'Country:', 'Longitude:', 'Latitude:'])

for index, row in enumerate(filtered_response):

    geocode_string = f'{row.dimension_values[0].value}, {row.dimension_values[1].value}, {row.dimension_values[2].value}'

    cities_df.iloc[index] = [
      int(row.metric_values[0].value),
      row.dimension_values[0].value,
      row.dimension_values[1].value,
      row.dimension_values[2].value,
     float(geocode(geocode_string)[0]['location']['x']),
     float(geocode(geocode_string)[0]['location']['y'])
    ]
    
cities_gdf = gpd.GeoDataFrame(cities_df, geometry=gpd.points_from_xy(cities_df["Longitude:"], cities_df["Latitude:"]))
cities_gdf.drop(columns=['Country:', 'Longitude:', "Latitude:"], inplace=True)

In [36]:
gdfToFeatureService(cities_gdf, 'LPT At Work IP Address Cities')

### 🛠️ GDFToHostedFeatureService Function Running: *(Created By Ben Beattie)*

Starting GDFToHostedTable          

✅ ***LPT_At_Work_IP_Address_Cities** has been updated...*